# Ensemble Learning

## Initial Imports

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter
from matplotlib import pyplot as plt

In [3]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from imblearn.ensemble import EasyEnsembleClassifier


## Read the CSV and Perform Basic Data Cleaning

In [4]:
# Load the data
file_path = Path('Resources/LoanStats_2019Q1.csv')
df = pd.read_csv(file_path)

# Preview the data
df

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500,0.1719,375.35,RENT,66000.0,Source Verified,19-Mar,low_risk,n,27.24,...,85.7,100.0,0,0,65687,38199,2000,61987,N,N
1,25000,0.2000,929.09,MORTGAGE,105000.0,Verified,19-Mar,low_risk,n,20.23,...,91.2,50.0,1,0,271427,60641,41200,49197,N,N
2,20000,0.2000,529.88,MORTGAGE,56000.0,Verified,19-Mar,low_risk,n,24.26,...,66.7,50.0,0,0,60644,45684,7500,43144,N,N
3,10000,0.1640,353.55,RENT,92000.0,Verified,19-Mar,low_risk,n,31.44,...,100.0,50.0,1,0,99506,68784,19700,76506,N,N
4,22000,0.1474,520.39,MORTGAGE,52000.0,Not Verified,19-Mar,low_risk,n,18.76,...,100.0,0.0,0,0,219750,25919,27600,20000,N,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68812,10000,0.1502,346.76,RENT,26000.0,Source Verified,19-Jan,low_risk,n,9.60,...,80.0,0.0,0,0,20625,6798,11300,5425,N,N
68813,12000,0.2727,368.37,RENT,63000.0,Not Verified,19-Jan,low_risk,n,29.07,...,96.2,0.0,0,0,87939,60350,13500,62939,N,N
68814,5000,0.1992,185.62,MORTGAGE,52000.0,Source Verified,19-Jan,low_risk,n,14.86,...,100.0,0.0,1,0,30592,18611,3600,18492,N,N
68815,40000,0.0646,1225.24,MORTGAGE,520000.0,Verified,19-Jan,low_risk,n,9.96,...,98.2,12.5,0,0,1033574,95958,100800,78634,N,N


## Split the Data into Training and Testing

In [5]:
# Create our feature
le = df.copy()
label_encoder = LabelEncoder()
#label_encoder.fit(X["home_ownership"])
#le["home_ownership_le"] = label_encoder.transform(X["home_ownership"])
label_encoder.fit(le["verification_status"])
le["verification_status_le"] = label_encoder.transform(le["verification_status"])
label_encoder.fit(le["loan_status"])
le["loan_status_le"] = label_encoder.transform(le["loan_status"])
label_encoder.fit(le["pymnt_plan"])
le["pymnt_plan_le"] = label_encoder.transform(le["pymnt_plan"])
label_encoder.fit(le["hardship_flag"])
le["hardship_flag_le"] = label_encoder.transform(le["hardship_flag"])
label_encoder.fit(le["debt_settlement_flag"])
le["debt_settlement_flag_le"] = label_encoder.transform(le["debt_settlement_flag"])
label_encoder.fit(le["application_type"])
le["application_type_le"] = label_encoder.transform(le["application_type"])
label_encoder.fit(le["initial_list_status"])
le["initial_list_status_le"] = label_encoder.transform(le["initial_list_status"])

X = pd.get_dummies(le, columns=["home_ownership"])


# Create our target
y = X["loan_status_le"]


X.drop(columns=["loan_status_le", "issue_d", "initial_list_status", "next_pymnt_d",  "verification_status", "loan_status", "pymnt_plan", "hardship_flag", "debt_settlement_flag", "issue_d", "application_type"], axis=1, inplace=True)
X.head()

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,verification_status_le,pymnt_plan_le,hardship_flag_le,debt_settlement_flag_le,application_type_le,initial_list_status_le,home_ownership_ANY,home_ownership_MORTGAGE,home_ownership_OWN,home_ownership_RENT
0,10500,0.1719,375.35,66000.0,27.24,0,0,8,0,1609,...,1,0,0,0,0,1,0,0,0,1
1,25000,0.2000,929.09,105000.0,20.23,0,0,17,1,18368,...,2,0,0,0,0,1,0,1,0,0
2,20000,0.2000,529.88,56000.0,24.26,0,0,8,0,13247,...,2,0,0,0,0,1,0,1,0,0
3,10000,0.1640,353.55,92000.0,31.44,0,1,10,1,17996,...,2,0,0,0,0,1,0,0,0,1
4,22000,0.1474,520.39,52000.0,18.76,0,1,14,0,9091,...,0,0,0,0,0,1,0,1,0,0


In [6]:
X.describe()

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,verification_status_le,pymnt_plan_le,hardship_flag_le,debt_settlement_flag_le,application_type_le,initial_list_status_le,home_ownership_ANY,home_ownership_MORTGAGE,home_ownership_OWN,home_ownership_RENT
count,68817.000000,68817.000000,68817.000000,6.881700e+04,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,...,68817.000000,68817.0,68817.0,68817.0,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000
mean,16677.594562,0.127718,480.652863,8.821371e+04,21.778153,0.217766,0.497697,12.587340,0.126030,17604.142828,...,0.669994,0.0,0.0,0.0,0.139660,0.876121,0.009285,0.526309,0.106747,0.357659
std,10277.348590,0.048130,288.062432,1.155800e+05,20.199244,0.718367,0.758122,6.022869,0.336797,21835.880400,...,0.719105,0.0,0.0,0.0,0.346637,0.329446,0.095914,0.499311,0.308793,0.479314
min,1000.000000,0.060000,30.890000,4.000000e+01,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,...,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,9000.000000,0.088100,265.730000,5.000000e+04,13.890000,0.000000,0.000000,8.000000,0.000000,6293.000000,...,0.000000,0.0,0.0,0.0,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000
50%,15000.000000,0.118000,404.560000,7.300000e+04,19.760000,0.000000,0.000000,11.000000,0.000000,12068.000000,...,1.000000,0.0,0.0,0.0,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000
75%,24000.000000,0.155700,648.100000,1.040000e+05,26.660000,0.000000,1.000000,16.000000,0.000000,21735.000000,...,1.000000,0.0,0.0,0.0,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000
max,40000.000000,0.308400,1676.230000,8.797500e+06,999.000000,18.000000,5.000000,72.000000,4.000000,587191.000000,...,2.000000,0.0,0.0,0.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [7]:
# Check the balance of our target values
# 1= low risk
y.value_counts()

1    68470
0      347
Name: loan_status_le, dtype: int64

In [8]:
# Split the X and y into X_train, X_test, y_train, y_test
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)
X_train.head()

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,verification_status_le,pymnt_plan_le,hardship_flag_le,debt_settlement_flag_le,application_type_le,initial_list_status_le,home_ownership_ANY,home_ownership_MORTGAGE,home_ownership_OWN,home_ownership_RENT
3903,1600,0.0881,50.74,35964.0,24.12,0,0,20,0,1156,...,0,0,0,0,0,0,0,0,1,0
28390,9000,0.0756,280.21,41000.0,16.89,1,0,16,0,11157,...,0,0,0,0,0,1,0,0,1,0
15470,10000,0.1033,214.10,112000.0,17.75,0,0,14,0,16904,...,2,0,0,0,0,1,0,1,0,0
4279,36000,0.1033,1167.21,120000.0,19.95,0,1,13,0,63248,...,0,0,0,0,0,1,0,0,0,1
57514,18000,0.0881,570.81,51000.0,19.11,1,0,16,0,17586,...,2,0,0,0,0,1,0,1,0,0


## Data Pre-Processing

Scale the training and testing data using the `StandardScaler` from `sklearn`. Remember that when scaling the data, you only scale the features data (`X_train` and `X_testing`).

In [9]:
# Create the StandardScaler instance
scaler = StandardScaler()

In [10]:
# Fit the Standard Scaler with the training data
# When fitting scaling functions, only train on the training dataset
X_scaler = scaler.fit(X_train)


In [11]:
# Scale the training and testing data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Ensemble Learners

In this section, I will compare two ensemble algorithms to determine which algorithm results in the best performance. You will train a Balanced Random Forest Classifier and an Easy Ensemble classifier . For each algorithm, I will complete the folliowing steps:

1. Train the model using the training data. 
2. Calculate the balanced accuracy score from sklearn.metrics.
3. Display the confusion matrix from sklearn.metrics.
4. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.
5. For the Balanced Random Forest Classifier only, print the feature importance sorted in descending order (most important feature to least important) along with the feature score

Note: I will use a random state of 1 for each algorithm to ensure consistency between tests

### Balanced Random Forest Classifier

In [13]:
# Resample the training data with the BalancedRandomForestClassifier
rf_model = RandomForestClassifier(n_estimators=500, random_state=1)
rf_model = rf_model.fit(X_train_scaled, y_train)
predictions = rf_model.predict(X_test_scaled)

In [14]:
# Calculated the balanced accuracy score

balanced_accuracy_score(y_test, predictions)

0.6779839583584177

In [15]:
# Display the confusion matrix
confusion_matrix(y_test, predictions)

array([[   36,    65],
       [    8, 17096]])

In [16]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, predictions))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.82      0.36      1.00      0.50      0.60      0.33       101
          1       1.00      1.00      0.36      1.00      0.60      0.38     17104

avg / total       1.00      1.00      0.36      0.99      0.60      0.38     17205



In [17]:
# List the features sorted in descending order by feature importance
pd.DataFrame({'Importance': rf_model.feature_importances_}, index=X.columns).sort_values(by='Importance', ascending=False)

,Importance
total_rec_prncp,0.066892
total_pymnt_inv,0.064004
total_pymnt,0.063093
total_rec_int,0.061073
last_pymnt_amnt,0.057519
...,...
hardship_flag_le,0.000000
debt_settlement_flag_le,0.000000
recoveries,0.000000
delinq_amnt,0.000000


### Easy Ensemble Classifier

In [18]:
# Train the Classifier
eec = EasyEnsembleClassifier(random_state=1)
eec.fit(X_train, y_train) 

predictions = eec.predict(X_test_scaled)


In [19]:
# Calculated the balanced accuracy score
balanced_accuracy_score(y_test, predictions)  

0.5

In [20]:
# Display the confusion matrix
confusion_matrix(y_test, predictions)

array([[  101,     0],
       [17104,     0]])

In [21]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, predictions))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.01      1.00      0.00      0.01      0.00      0.00       101
          1       0.00      0.00      1.00      0.00      0.00      0.00     17104

avg / total       0.00      0.01      0.99      0.00      0.00      0.00     17205



### Final Questions

1. Which model had the best balanced accuracy score?

    The Random Forest Classifier(RFC) had the best balanced accuracy score(BAC) of 0.67 as opposed to the Easy Ensemble Classifier(EEC) BAC of 0.5. 

2. Which model had the best recall score?

    The RFC had a perfect recall of True Positives(TP) which are accuratley predicting low risk loans, while only accurately predicting .36% of high risk loans. 
    The EEC had a perfect recall of True Negative(TN)  which are accuratley predicting high risk loans, while not accurately predicitng any low risk loans.
    

3. Which model had the best geometric mean score?
    The RFC geometric mean score is 0.60 is better than the EEC score. 
    The EEC geometric mean score is 0 because it did not identify 1 of the classes.
    

4. What are the top three features?

    The top 3 features in order are 1. total_rec_prncp 2. total_pymnt_inv	3.total_pymnt